In [ ]:
from google.colab import files
uploaded = files.upload()

Saving output_with_census_codes.csv to output_with_census_codes (1).csv


In [ ]:
import pandas as pd
import requests
import time

In [ ]:
CENSUS_API_KEY = "77717138b7fca62f0286916d6ea887a559a8ba63"

In [ ]:
# Load your data with state, county, tract, and block codes
df = pd.read_csv("output_with_census_codes.csv")  # Replace with your file path

In [ ]:
# Convert codes to strings to avoid decimals
df['State_FIPS'] = df['State_FIPS'].astype(str).str.zfill(2)     # Ensure state code is 2 digits
df['County_FIPS'] = df['County_FIPS'].astype(str).str.zfill(3)   # Ensure county code is 3 digits
df['Tract_Code'] = df['Tract_Code'].astype(str).str.zfill(6)     # Ensure tract code is 6 digits
df['Block_Code'] = df['Block_Code'].astype(str).str.zfill(4)     # Ensure block code is 4 digits

In [ ]:
# Add columns for tract and block-level race demographics
df['Total_Population_Tract'] = None
df['White_Alone_Tract'] = None
df['Black_Alone_Tract'] = None
df['American_Indian_Alone_Tract'] = None
df['Asian_Alone_Tract'] = None
df['Pacific_Islander_Alone_Tract'] = None
df['Other_Race_Alone_Tract'] = None
df['Two_or_More_Races_Tract'] = None

df['Total_Population_Block'] = None
df['White_Alone_Block'] = None
df['Black_Alone_Block'] = None
df['American_Indian_Alone_Block'] = None
df['Asian_Alone_Block'] = None
df['Pacific_Islander_Alone_Block'] = None
df['Other_Race_Alone_Block'] = None
df['Two_or_More_Races_Block'] = None

In [ ]:
# Function to get racial demographics for each geographic level
def get_race_data(state, county, tract, block=None):
    # Determine if we're getting data for a block or tract level
    if block:
        # Block-level data request
        url = f"https://api.census.gov/data/2020/dec/pl?get=P1_001N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,P1_009N&for=block:{block}&in=state:{state}+county:{county}+tract:{tract}&key={CENSUS_API_KEY}"
    else:
        # Tract-level data request
        url = f"https://api.census.gov/data/2020/dec/pl?get=P1_001N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,P1_009N&for=tract:{tract}&in=state:{state}+county:{county}&key={CENSUS_API_KEY}"

    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            # Extract race demographic data
            total_pop = data[1][0]
            white_alone = data[1][1]
            black_alone = data[1][2]
            american_indian_alone = data[1][3]
            asian_alone = data[1][4]
            pacific_islander_alone = data[1][5]
            other_race_alone = data[1][6]
            two_or_more_races = data[1][7]
            return total_pop, white_alone, black_alone, american_indian_alone, asian_alone, pacific_islander_alone, other_race_alone, two_or_more_races
        except (IndexError, ValueError):
            return None, None, None, None, None, None, None, None
    else:
        print(f"Failed to retrieve data for state:{state}, county:{county}, tract:{tract}, block:{block} - Status Code: {response.status_code}")
        return None, None, None, None, None, None, None, None


In [ ]:
# Apply the function to each row in the DataFrame
for index, row in df.iterrows():
    if pd.notna(row['State_FIPS']) and pd.notna(row['County_FIPS']) and pd.notna(row['Tract_Code']) and pd.notna(row['Block_Code']):
        state = row['State_FIPS']
        county = row['County_FIPS']
        tract = row['Tract_Code']
        block = row['Block_Code']

        # Get block-level data
        total_pop_blk, white_alone_blk, black_alone_blk, american_indian_alone_blk, asian_alone_blk, pacific_islander_alone_blk, other_race_alone_blk, two_or_more_races_blk = get_race_data(state, county, tract, block)
        df.at[index, 'Total_Population_Block'] = total_pop_blk
        df.at[index, 'White_Alone_Block'] = white_alone_blk
        df.at[index, 'Black_Alone_Block'] = black_alone_blk
        df.at[index, 'American_Indian_Alone_Block'] = american_indian_alone_blk
        df.at[index, 'Asian_Alone_Block'] = asian_alone_blk
        df.at[index, 'Pacific_Islander_Alone_Block'] = pacific_islander_alone_blk
        df.at[index, 'Other_Race_Alone_Block'] = other_race_alone_blk
        df.at[index, 'Two_or_More_Races_Block'] = two_or_more_races_blk

        # Get tract-level data (no block parameter)
        total_pop_trc, white_alone_trc, black_alone_trc, american_indian_alone_trc, asian_alone_trc, pacific_islander_alone_trc, other_race_alone_trc, two_or_more_races_trc = get_race_data(state, county, tract)
        df.at[index, 'Total_Population_Tract'] = total_pop_trc
        df.at[index, 'White_Alone_Tract'] = white_alone_trc
        df.at[index, 'Black_Alone_Tract'] = black_alone_trc
        df.at[index, 'American_Indian_Alone_Tract'] = american_indian_alone_trc
        df.at[index, 'Asian_Alone_Tract'] = asian_alone_trc
        df.at[index, 'Pacific_Islander_Alone_Tract'] = pacific_islander_alone_trc
        df.at[index, 'Other_Race_Alone_Tract'] = other_race_alone_trc
        df.at[index, 'Two_or_More_Races_Tract'] = two_or_more_races_trc

In [ ]:
df.to_csv("output_with_racial_demographics_block_and_tract.csv", index=False)


In [ ]:
from google.colab import files
files.download("output_with_racial_demographics_block_and_tract.csv")